---
# 1 Introducción

El siguiente cuaderno realiza el binomio de Newton[1] entre los componentes de dos vectores. El ejercicio se resuelve usando CPU-GPU para ver los tiempos de respuesta de cada tipo de ejecución.

El binomio de Newton esta dado por la formnula:

 <center> $(a+b)^n$ </center>

**Donde:**

$a$ y $b$ son cada uno de los componentes de los vectores.

$n$ es el parámetro potencia que solicita el código.

Este cuaderno resuelvo hasta el binomio de potencia 4.

Tipos de binomio:

Potencia: 0

$(a+b)^0 = 1$

Potencia: 1

$(a+b)^1 = a+b$

Potencia: 2

$(a+b)^2 = a^2+2a*b+b^2$

Potencia: 3

$(a+b)^3 = a^3+3a^2*b+3a*b^2+b^3$

Potencia: 4

$(a+b)^4 = a^4+4a^3*b+6a^2*b^2+4a*b^3+b^4$


Estos ejercicios están resueltos en lenguaje Python[2] en la plataforma Colab[3]

---
# 2 Armado del ambiente
Instala en el cuaderno el módulo CUDA de Python para el correcto funcionamiento del item 3.2 donde el codigo se ejecuta en CPU-GPU.

In [ ]:
!pip install pycuda

---
# 3 Desarrollo


In [ ]:
# --------------------------------------------
#@title 3.1 Parámetros de ejecución de CPU-GPU { vertical-output: true }

class ParametroException(Exception):
  #Ocurre cuando los parametros ingresados son incorrectos.
  def __init__(self,*args,**kwargs):
    Exception.__init__(self,*args,**kwargs)

try:
  cantidad =   300#@param {type: "number"}
  potencia =   3#@param {type: "number"}

  # --------------------------------------------
  if potencia < 0 or potencia > 4 or type(potencia) != type(1):
    raise TypeError
  if cantidad < 0:
    raise TypeError

  from datetime import datetime

  tiempo_total = datetime.now()
  try:
    import sys
    import pycuda.driver as cuda
    import pycuda.autoinit
    from pycuda.compiler import SourceModule
  except:
    print("No se ejecutó el Item 2 - Armado de ambiente.")
    raise NameError("Item 2 no ejecutado")

  import numpy

  # --------------------------------------------
  # Definición de función que transforma el tiempo en  milisegundos 
  tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0

  x_cpu = numpy.random.randn(cantidad)
  x_cpu = x_cpu.astype(numpy.float32())

  y_cpu = numpy.random.randn(cantidad)
  y_cpu = y_cpu.astype(numpy.float32()) 

  res_cpu = numpy.empty_like (x_cpu)

  #CPU - reservo la memoria GPU
  x_gpu = cuda.mem_alloc(x_cpu.nbytes)
  y_gpu = cuda.mem_alloc(y_cpu.nbytes)

  #GPU - Copio la memoria al GPU
  cuda.memcpy_htod(x_gpu, x_cpu)
  cuda.memcpy_htod(y_gpu, y_cpu)


  #CPU - Defino la función kernel que se ejecutará en GPU.
  module = SourceModule("""

  __global__ void kernel_binomio(int n,int pot, float *X , float *Y)
  {
      int idx = threadIdx.x + blockIdx.x*blockDim.x;
      if( idx < n){

        switch(pot)
        {
            case 0:
              Y[idx] = 1;
            break;
            case 1:
              Y[idx] = X[idx] + Y[idx];
            break;
            case 2:
              Y[idx] = pow(X[idx],2) + 2*X[idx]*Y[idx]+pow(Y[idx],2);
            break;
            case 3:
              Y[idx] = pow(X[idx],3) + 3*pow(X[idx],2)*Y[idx]+3*X[idx]*pow(Y[idx],2)+pow(Y[idx],3);
            break;
            case 4:
              Y[idx] = pow(X[idx],4) + 4*pow(X[idx],3)*Y[idx]+6*pow(X[idx],2)*pow(Y[idx],2)+ 4*X[idx]*pow(Y[idx],3)+pow(Y[idx],4);
            break;
        }
      }   
  }
  """)
  #CPU - genero la función kernel
  kernel = module.get_function("kernel_binomio")

  tiempo_gpu = datetime.now()

  # GPU - Ejecuta el kernel.
  dim_hilo = 256
  dim_bloque = numpy.int( (cantidad+dim_hilo-1) / dim_hilo )

  kernel( numpy.int32(cantidad),numpy.int32(potencia), x_gpu, y_gpu, block=( dim_hilo, 1, 1 ),grid=(dim_bloque, 1,1) )

  tiempo_gpu = datetime.now() - tiempo_gpu


  # GPU - Copio el resultado desde la memoria GPU.
  cuda.memcpy_dtoh(res_cpu, y_gpu)

  """
  # CPU - Informo el resutlado.
  print( "------------------------------------")
  print( "X: " )
  print( x_cpu )
  print( "------------------------------------")
  print( "Y: " )
  print( y_cpu )
  print( "------------------------------------")
  print( "Resultado: " )
  print( res_cpu )
  """

  tiempo_total = datetime.now() - tiempo_total

  print( "Cantidad de elementos: ", cantidad)
  print( "Thread x: ", dim_hilo, ", Bloque x:", dim_bloque )
  print("Tiempo CPU: ", tiempo_en_ms( tiempo_total ), "[ms]" )
  print("Tiempo GPU: ", tiempo_en_ms( tiempo_gpu   ), "[ms]" )

except TypeError as err:
  print("Los parametros ingresados son incorrectos.")
  print("Cantidad debe ser un numero entero positivo.")
  print("Potencia debe ser un numero entero comprendido entre 0 y 4.")
  print(err)
except NameError as err:
  print(err)
except Exception as err:
  print("Ocurrió un error inesperado.")
  print(type(err))
  print(err)

---
#4 Tabla de pasos
Tabla de pasos de la ejecución del programa CPU-GPU:

 Procesador | Funciòn | Detalle
------------|---------|----------
CPU      |  Custom Exception                | Se crea una excepcion customizada
CPU      |  @param                | Lectura del tamaño de vectores desde Colab.
CPU      |  validación                | Se validan los parametros ingresados.
CPU      |  import                | Importa los módulos para funcionar.
CPU      |  datetime.now()        | Toma el tiempo actual.
CPU      |  numpy.random.randn( Cantidad ) | Inicializa los vectores X, Y y Res.
**GPU**  |  cuda.mem_alloc()      | Reserva la memoria en GPU.
**GPU**  |  cuda.memcpy_htod()    | Copia las memorias desde el CPU al GPU.
CPU      |  SourceModule()        | Define el código del kernel 
CPU      |  module.get_function() | Genera la función del kernel GPU
CPU      |  dim_tx/dim_bx         | Calcula las dimensiones.
**GPU**  |  kernel()              | Ejecuta el kernel en GPU
CPU      |  cuda.memcpy_dtoh( )   | Copia el resultado desde GPU a CPU vector Res.
CPU      |  print()               | Informo los resultados.

---
# 5 Conclusiones

Como se puede ver en las ejecuciones, para poco cálculo la ejecución secuencial en CPU es más rápida ya que la ejecución CPU-GPU tiene que hacer la reserva de memoria en el GPU, copiar los datos a trabajar y despues copiar otra vez los resultados al vector correspondiente, por lo que se traduce en un overhead enorme y por los pocos datos a procesar no tiene sentido.

Pero si la carga de datos a procesar es muy grande la diferencia entre trabajar secuencialmente con trabajar con los threads del GPU de forma paralela es muy amplia y ahí si tiene sentido y da una respuesta muchisimo más rapida trabajar con el GPU.

---
# 6 Bibliografía

[1] Binomio de Newtom: [Wiki](https://es.wikipedia.org/wiki/Teorema_del_binomio)

[2] Introducción a Python: [Página Colab](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/Python_Basico.ipynb) 

[3] MARKDOWN SYNTAX Colab: [PDF](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/markdown-cheatsheet-online.pdf)